# ĐỒ ÁN CUỐI KỲ (LTKHDL - CQ2017/2)
### GVHD: TS. Trần Trung Kiên

Họ và tên sinh viên: Nguyễn Hữu Huân

MSSV: 1712466

------

## Import

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

---

## Lấy dữ liệu

Dữ liệu mà em sử dụng ở đây được lấy từ [dataset của Yelp trên Kaggle](https://www.kaggle.com/yelp-dataset/yelp-dataset). Yelp là một công ty công nghệ lớn ở Mỹ với sản phẩm nổi bật là Yelp website và Yelp app cho phép người dùng tìm kiếm, đánh giá, bình luận, đặt chỗ dịch vụ của các cở sở kinh doanh. 

Khi tải dataset này về đồng nghĩa với việc đã chấp nhận điều khoản sử dụng (một file pdf khá dài) được viết bởi Yelp dành riêng cho dataset này ví dụ như là chỉ được sử dụng cho mục đích học tập giáo dục,...

Vì dataset này khá lớn (tầm 10GB) và chủ đề mà em quan tâm ở đây chỉ là nhà hàng, món ăn nên em chỉ sử dụng 2 file json là: "yelp_academic_dataset_business.json" và "yelp_academic_dataset_review.json" trong số 5 file của dataset. File "yelp_academic_dataset_business.json" chứa thông tin của các cơ sở kinh doanh, còn file "yelp_academic_dataset_review.json" chứa thông tin đánh giá của người dùng đối với cơ sở kinh doanh đó.  

-----

## Khám phá dữ liệu

### Data Description

Phần mô tả dữ liệu này Yelp không cung cấp trên Kaggle mà để ở trang chủ của mình [ở đây](https://www.yelp.com/dataset/documentation/main). Em có dịch sơ lại như sau:

- "yelp_academic_dataset_review.json":
    - **business_id**: mã id của doanh nghiệp (string, 22 chữ số và duy nhất)
    - **name**: tên doanh nghiệp (string)
    - **address**: địa chỉ (string)
    - **city**: thành phố (string)
    - **state**: mã bang  (string, 2 kí tự)
    - **postal_code**: mã bưu chính (string)
    - **latitude**: vĩ độ (float)
    - **longitude**: kinh độ (float)
    - **stars**: sao trung bình (float, làm tròn 0.5)
    - **review_count**: số lượng review (int)
    - **is_open**: có còn hoạt động hay không (int, 0: là đã đóng cửa và 1: còn hoạt động)
    - **attribute**: một số thông tin về doanh nghiệp (object : đậu xe như thế nào, thanh toán thẻ được không,..)
    - **categories**: danh sách các loại của doanh nghiệp (array of string)
    - **hours**: giở mở cửa (object: Thứ Hai từ mấy giờ tới mấy giờ, Thứ Ba,...)
- "yelp_academic_dataset_review.json":
    - **user_id**: mã id người dùng (string, 22 chữ số và duy nhất)
    - **text**: nội dung review (string)
    - **date**: ngày review (string, format là yyyy-mm-dd HH:MM:SS)
    - **review_id**: mã id của review (string, 22 chữ số và duy nhất)
    - **business_id**: mã doanh nghiệp được review (string, 22 chữ số và duy nhất)
    - **funny**: độ hài hước của review này được vote (int)
    - **cool**: độ "ngầu" của review này được vote (int)
    - **useful**: độ có ích của review được vote (int)
    - **stars**: số sao review (int)

### Đọc dữ liệu vào DataFrame

Khá may mắn là mặc dù các file này là json nhưng cấu trúc nó khá "ngay ngắn" và có thể đọc thành DataFrame bằng hàm `pd.read_json()`. Nhưng cũng khá xui vì file về reviews khá nặng (6GB!), ban đầu em cũng thử cố đọc file này bằng cách định nghĩa các cột trước khi đọc và chia thành đọc các 'chunk' nhỏ, tuy nhiên máy vẫn bị "đơ". Để tiết kiệm thời gian thì em đã lên Kaggle tải file ["reviews.csv"](https://www.kaggle.com/darshank2019/review) mà người khác đã convert trước đó.

In [2]:
business_df = pd.read_json('data/yelp_academic_dataset_business.json', lines=True)

In [64]:
type(business_df.iloc[0,12])

str

In [4]:
business_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,10913 Bailey Rd,Cornelius,NC,28031,35.462724,-80.852612,3.5,36,1,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Active Life, Gun/Rifle Ranges, Guns & Ammo, Sh...","{'Monday': '10:0-18:0', 'Tuesday': '11:0-20:0'..."
1,Yzvjg0SayhoZgCljUJRF9Q,"Carlos Santo, NMD","8880 E Via Linda, Ste 107",Scottsdale,AZ,85258,33.569404,-111.890264,5.0,4,1,"{'GoodForKids': 'True', 'ByAppointmentOnly': '...","Health & Medical, Fitness & Instruction, Yoga,...",None
2,XNoUzKckATkOD1hP6vghZg,Felinus,3554 Rue Notre-Dame O,Montreal,QC,H4C 1P4,45.479984,-73.580070,5.0,5,1,None,"Pets, Pet Services, Pet Groomers",None
3,6OAZjbxqM5ol29BuHsil3w,Nevada House of Hose,1015 Sharp Cir,North Las Vegas,NV,89030,36.219728,-115.127725,2.5,3,0,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Hardware Stores, Home Services, Building Suppl...","{'Monday': '7:0-16:0', 'Tuesday': '7:0-16:0', ..."
4,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,4827 E Downing Cir,Mesa,AZ,85205,33.428065,-111.726648,4.5,26,1,"{'BusinessAcceptsCreditCards': 'True', 'ByAppo...","Home Services, Plumbing, Electricians, Handyma...","{'Monday': '0:0-0:0', 'Tuesday': '9:0-16:0', '..."


 Vì file csv này là của phiên bản reviews cách đây 1 năm (phiên bản mới nhất cách đây 5 tháng) nên dung lượng chỉ còn khoảng 4GB. Nhưng 4GB vẫn là con số khá lớn, cho nên sau khi đọc data description em đã bỏ đọc một số cột em cảm thấy sẽ không cần dùng tới (theo tiêu chí bài này của em):"user_id","useful", "text", "cool", "funny" để việc đọc được diễn ra "suông sẻ" và nhanh hơn.

In [9]:
reviews_df = pd.read_csv('data/yelp_academic_dataset_review.csv', usecols=['date','review_id','business_id','stars'])

In [10]:
reviews_df.head()

,date,review_id,business_id,stars
0,b'2013-05-07 04:34:36',b'Q1sbwvVQXV2734tPgoKj4Q',b'ujmEBvifdJM6h6RLv4wQIg',1.0
1,b'2017-01-14 21:30:33',b'GJXCdrto3ASJOqKeVWPi6Q',b'NZnhc2sEQy3RmzKTZnqtwQ',5.0
2,b'2016-11-09 20:09:03',b'2TzJjDVDEuAW6MR5Vuc1ug',b'WTqjgwHlXbSFevF32_DJVw',5.0
3,b'2018-01-09 20:56:38',b'yi0R0Ugj_xUx_Nek0-_Qig',b'ikCg8xy5JIg_NGPx-MSIDA',5.0
4,b'2018-01-30 23:07:38',b'11a8sVPMUFtaC7_ABRkmtw',b'b1b1eb3uo-w561D0ZfCEiQ',1.0


### Dữ liệu gồm bao nhiêu dòng, bao nhiêu cột?

Quan sát kích thước của 2 DataFrame

In [46]:
business_df.shape

(209393, 14)

In [47]:
reviews_df.shape

(6685900, 4)

Dữ liệu khá bự 😥 14 cột, 209393 dòng với `business_df`, mỗi dòng là một doanh nghiệp. 6 cột 6685900 dòng với `reviews_df`, mỗi dòng là một review.

### Mỗi dòng, cột có ý nghĩa gì?

Ý nghĩa của từng cột thì đã có ở phần **Data Description** phía trên. Ý nghĩa mỗi dòng của `business_df` là một cửa hàng kinh doanh, còn mỗi dòng của `reviews_df` là một review của người dùng lên cửa hàng nào đó.

### Mỗi cột có dtypes là gì? Dữ liệu có thiếu không? Ý nghĩa là sao?

🏪 DataFrame business_df  

In [15]:
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209393 entries, 0 to 209392
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   business_id   209393 non-null  object 
 1   name          209393 non-null  object 
 2   address       209393 non-null  object 
 3   city          209393 non-null  object 
 4   state         209393 non-null  object 
 5   postal_code   209393 non-null  object 
 6   latitude      209393 non-null  float64
 7   longitude     209393 non-null  float64
 8   stars         209393 non-null  float64
 9   review_count  209393 non-null  int64  
 10  is_open       209393 non-null  int64  
 11  attributes    180348 non-null  object 
 12  categories    208869 non-null  object 
 13  hours         164550 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 22.4+ MB


**Một số thông tin về dtype và null values:**

Ở đây để kiểm tra kỹ hơn về dtype của một vài cột object thì em sẽ tái sử dụng lại hàm `open_object_dtype()` (BT03-Pandas).

In [65]:
def open_object_dtype(s):
    dtypes = set()
    for value in s.values:
            dtypes.add(type(value))
    return dtypes

- Có thể thấy **postal_code** là string thì hơi lạ, nhưng sau khi tìm hiểu thì postal code của Mỹ có bao gồm cả chữ cái (khác VN)

In [66]:
open_object_dtype(business_df.postal_code)

{str}

In [24]:
business_df.postal_code.sample(3)

78469       44060
134499    M5J 2N3
46601       28105
Name: postal_code, dtype: object

- **hours** và **attribute** ở đây là object (cụ thể là json object hay dict của python và NoneType (của các null values))

In [67]:
open_object_dtype(business_df.hours)

{NoneType, dict}

In [68]:
open_object_dtype(business_df.attributes)

{NoneType, dict}

In [63]:
business_df.hours.sample(3)

61902                                                  None
17388     {'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...
204480    {'Tuesday': '10:0-19:0', 'Wednesday': '10:0-19...
Name: hours, dtype: object

In [25]:
business_df.attributes.sample(3)

59793     {'BusinessAcceptsCreditCards': 'True', 'Busine...
133772    {'BusinessAcceptsCreditCards': 'True', 'ByAppo...
55702     {'RestaurantsPriceRange2': '2', 'Caters': 'Tru...
Name: attributes, dtype: object

- Các cột chứa giá trị thiếu bao gồm: **attributes**, **categories**, **hours**. 2 cái còn lại thì không sao nhưng **categories** thiếu thì hơi khó khăn trong việc nhận diện loại dịch vụ. Vì vậy em sẽ drop những dòng thiếu cột này.

In [48]:
business_df.dropna(axis=0, subset=['categories'], inplace=True)

- **categories** là object (str) được lưu theo kiểu liệt kê: A, B, C. Vì chủ đề ban đầu em chọn là về nhà hàng, nên với miền các giá trị như hiện tại của **categories** sẽ bao gồm các record em không cần tới. Do đó, em sẽ chỉ giữ lại những record có **categories** bao gồm nhà hàng.

In [69]:
open_object_dtype(business_df.categories)

{str}

In [26]:
business_df.categories.sample(3)

120148    Food, Cafes, French, Bakeries, Restaurants, Bi...
10476     Active Life, Cycling Classes, Trainers, Yoga, ...
63893                  Beer, Wine & Spirits, Food, Shopping
Name: categories, dtype: object

In [43]:
# show the "range" of catergories 
business_df.assign(categories = business_df.categories
                         .str.split(', ')).explode('categories').categories.value_counts()  # all kinds of categories in value-count order

Restaurants        63944
Shopping           34644
Food               32991
Home Services      22487
Beauty & Spas      20520
                   ...  
Japanese Sweets        1
Halfway Houses         1
Backshop               1
Minho                  1
Calabrian              1
Name: categories, Length: 1336, dtype: int64

In [49]:
business_df = business_df[business_df.categories.str.contains('Restaurants')]

In [51]:
business_df.shape

(63944, 14)

- cột **is_open** là kiểu numeric, nhưng thực chất là categorical được viết dưới dạng số: 0 - đã đóng cửa, 1 - còn hoạt động. Em chỉ giữ lại những nhà hàng còn mở cửa thôi.

In [55]:
business_df = business_df[business_df.is_open == 1]

In [56]:
business_df.shape

(43965, 14)

📝 DataFrame reviews_df  

In [71]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6685900 entries, 0 to 6685899
Data columns (total 4 columns):
 #   Column       Dtype         
---  ------       -----         
 0   date         datetime64[ns]
 1   review_id    object        
 2   business_id  object        
 3   stars        float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 204.0+ MB


**Một số thông tin về dtype:**

In [72]:
print(open_object_dtype(reviews_df.review_id))
print(open_object_dtype(reviews_df.business_id))
print(open_object_dtype(reviews_df.date))

{<class 'str'>}
{<class 'str'>}


- Các cột object(string) chứa nội dung có vấn đề: cụ thể là dạng b'\<nội dung>\'. Lỗi này có thể là do người ghi file csv này, do đó em phải xử lý bỏ những kí tự thừa ra.

In [17]:
reviews_df.date = reviews_df.date.str.extract(r"b\'(.+)\'")

In [19]:
reviews_df.review_id = reviews_df.review_id.str.extract(r"b\'(.+)\'")

In [20]:
reviews_df.business_id = reviews_df.business_id.str.extract(r"b\'(.+)\'")

In [21]:
reviews_df.head()

,date,review_id,business_id,stars
0,2013-05-07 04:34:36,Q1sbwvVQXV2734tPgoKj4Q,ujmEBvifdJM6h6RLv4wQIg,1.0
1,2017-01-14 21:30:33,GJXCdrto3ASJOqKeVWPi6Q,NZnhc2sEQy3RmzKTZnqtwQ,5.0
2,2016-11-09 20:09:03,2TzJjDVDEuAW6MR5Vuc1ug,WTqjgwHlXbSFevF32_DJVw,5.0
3,2018-01-09 20:56:38,yi0R0Ugj_xUx_Nek0-_Qig,ikCg8xy5JIg_NGPx-MSIDA,5.0
4,2018-01-30 23:07:38,11a8sVPMUFtaC7_ABRkmtw,b1b1eb3uo-w561D0ZfCEiQ,1.0


- Cột date cần phải đổi trừ object sang datetime

In [24]:
reviews_df['date'] = pd.to_datetime(reviews_df['date'], format='%Y-%m-%d %H:%M:%S')

----

## Đưa ra các câu hỏi cần trả lời

In [73]:
df1 = reviews_df.rename({'stars': 'review_stars'})
df2 = business_df
df = df1.merge(df2)
df

,date,review_id,business_id,stars,name,address,city,state,postal_code,latitude,longitude,review_count,is_open,attributes,categories,hours
0,2016-01-17 05:26:22,IByf6mVY0WA838XNSh-5MA,I4Nr-MVc26qWr08-S3Q1ow,4.0,Greenberg's Deli,3790 Las Vegas Blvd S,Las Vegas,NV,89136,36.102725,-115.174164,388,1,"{'RestaurantsAttire': 'u'casual'', 'BikeParkin...","Delis, Restaurants, Sandwiches, Pizza","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ..."
1,2018-08-14 12:34:37,FQNrnnmmxwdHyrQ84qkgHQ,I4Nr-MVc26qWr08-S3Q1ow,4.0,Greenberg's Deli,3790 Las Vegas Blvd S,Las Vegas,NV,89136,36.102725,-115.174164,388,1,"{'RestaurantsAttire': 'u'casual'', 'BikeParkin...","Delis, Restaurants, Sandwiches, Pizza","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ..."
2,2016-12-04 06:45:18,1lPKVO_l-B9O3Vv63ry6rw,I4Nr-MVc26qWr08-S3Q1ow,4.0,Greenberg's Deli,3790 Las Vegas Blvd S,Las Vegas,NV,89136,36.102725,-115.174164,388,1,"{'RestaurantsAttire': 'u'casual'', 'BikeParkin...","Delis, Restaurants, Sandwiches, Pizza","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ..."
3,2013-12-08 02:03:06,h-S9hbHrkV8rFegFqpdf_Q,I4Nr-MVc26qWr08-S3Q1ow,4.0,Greenberg's Deli,3790 Las Vegas Blvd S,Las Vegas,NV,89136,36.102725,-115.174164,388,1,"{'RestaurantsAttire': 'u'casual'', 'BikeParkin...","Delis, Restaurants, Sandwiches, Pizza","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ..."
4,2016-07-15 04:21:06,4rBiVfkJw86HwHiPfpg8cA,I4Nr-MVc26qWr08-S3Q1ow,4.0,Greenberg's Deli,3790 Las Vegas Blvd S,Las Vegas,NV,89136,36.102725,-115.174164,388,1,"{'RestaurantsAttire': 'u'casual'', 'BikeParkin...","Delis, Restaurants, Sandwiches, Pizza","{'Monday': '9:0-22:0', 'Tuesday': '9:0-22:0', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488697,2018-03-07 23:58:16,ItxihFlmqClYwNAFUbkaKg,ZsYOG1b786NtBVq_OssdTg,4.0,Calaveras,2040 Rue Beaubien E,Montréal,QC,H2G 1M4,45.545310,-73.594570,9,1,None,"Restaurants, Mexican","{'Tuesday': '17:0-23:0', 'Wednesday': '17:0-23..."
488698,2016-10-12 00:14:43,C6Fh0AFpSuWl7zKXduRQ_Q,n9PgccTPsLxgYL6ZCkxDMQ,3.0,Pittzarella,5832 Library Rd,Bethel Park,PA,15102,40.308755,-80.033243,11,1,"{'RestaurantsPriceRange2': '2', 'RestaurantsTa...","Pizza, Restaurants","{'Monday': '10:0-23:0', 'Tuesday': '10:0-23:0'..."
488699,2014-10-20 02:06:23,BOM0UHgL5660rs-RXCU7mg,3xI0ToitrAJkrUSH4gbmCA,2.0,Subway Restaurants,"4747 East Elliot Rd., Suite 33",Phoenix,AZ,85044,33.347499,-111.981221,14,1,"{'GoodForKids': 'True', 'WiFi': 'u'no'', 'Busi...","Restaurants, Sandwiches, Fast Food","{'Monday': '7:0-21:0', 'Tuesday': '7:0-21:0', ..."
488700,2012-04-23 20:58:49,EnMFnR1U0ZeGMChuKoM-rw,e_vguiiAy096g8-e0oQDDg,2.0,Subway,507 Saint Clair Avenue W,Toronto,ON,M6C 1A1,43.682907,-79.418504,5,1,"{'BusinessParking': '{'garage': False, 'street...","Restaurants, Sandwiches, Fast Food","{'Monday': '7:0-23:0', 'Tuesday': '7:0-23:0', ..."


Trước đó nhìn sơ xem th

------

## Tìm câu trả lời cho từng câu hỏi

-------

## Nhìn lại quá trình làm đồ án

---

## Tài liệu tham khảo